In [3]:
"""
Extract crawled_data/matches to new_lake/50k_matches.parquet and create check_download_player_id.parquet
"""
import pandas as pd
import json
import time
from pathlib import Path
import glob
from progress.bar import IncrementalBar
from datetime import datetime
import numpy as np
import sys
import requests


ROOT = f'D:\Python\hackathon'
BASE_PATH = f'D:\Python\hackathon\source'
SUB_FOLDER = "crawled_data/matches"
OUTPUT_FILE = "new_lake/50k_matches.parquet"
CHECK_FILE = "check_download_player_id.parquet"
# add my source
SOUCRE_PATH = str(Path(ROOT, "source"))
sys.path.append(SOUCRE_PATH)
from common import *

columns = [
    "match_id",
    "game_mode",
    "account_id01",
    "hero_id01",
    "isRadiant01",
    "account_id02",
    "hero_id02",
    "isRadiant02",
    "account_id03",
    "hero_id03",
    "isRadiant03",
    "account_id04",
    "hero_id04",
    "isRadiant04",
    "account_id05",
    "hero_id05",
    "isRadiant05",
    "account_id06",
    "hero_id06",
    "isRadiant06",
    "account_id07",
    "hero_id07",
    "isRadiant07",
    "account_id08",
    "hero_id08",
    "isRadiant08",
    "account_id09",
    "hero_id09",
    "isRadiant09",
    "account_id10",
    "hero_id10",
    "isRadiant10",
    "radiant_win",
    "leagueid",
    "region",
    "patch",
]
matches_path = str(Path(ROOT, SUB_FOLDER))
# print(matches_path)

match_list = glob.glob(f"{matches_path}\*\*\*\*.json")
# for (dir_path, dir_names, file_names) in os.walk(matches_path):
#     match_list.extend(file_names)
# print(len(match_list))

bar = IncrementalBar(
    "Extracting matches and players id...",
    max=len(match_list),
    suffix="%(index)d/%(max)d | %(elapsed_td)s",
)

# matches_id = [get_match_id(file) for file in match_list]
# print(len(match_id))
bu = 0
data = {}
for col in columns:
    data[col] = []
account_list = []
now = datetime.now()
crawl_time = int(np.rint(datetime.timestamp(now)))
error_files = []
for file in match_list:
    # print(file)
    # print(type(file))
    with open(file, "rb") as f:
        try:
            data_json = json.load(f)
            # ignore the match if this match dont have players.
            if "players" in data_json.keys():
                players_data = data_json["players"]
                for i in range(10):
                    num = str(i + 1).zfill(2)
                    if i < len(players_data):
                        data["account_id" + num].append(players_data[i]["account_id"])
                        data["hero_id" + num].append(players_data[i]["hero_id"])
                        data["isRadiant" + num].append(players_data[i]["isRadiant"])
                        account_list.append(
                            players_data[i]["account_id"]
                        )  # get player id to save
                    else:
                        data["account_id" + num].append(None)
                        data["hero_id" + num].append(None)
                        data["isRadiant" + num].append(None)
                for col in [
                    "match_id",
                    "game_mode",
                    "radiant_win",
                    "leagueid",
                    "region",
                    "patch",
                ]:
                    if col in data_json.keys():
                        data[col].append(data_json[col])
                    else:
                        data[col].append(None)
            else:
                bar.next()
                error_files.append(file)
                continue
            bar.next()
        except:
            error_files.append(file)
            bar.next()
            continue
bar.finish()


In [4]:

account_list1 = set(account_list)
account_list1.discard(None)
accounts = [*account_list1]
# print(len(accounts))
account_df = pd.DataFrame(accounts, columns=["player_id"])
account_df["time"] = crawl_time
account_df["downloaded"] = 0
# print("Error files:", error_files)




In [5]:
print(len(error_files))

3746


In [18]:
# reset check downloaded of error_files= 0
if error_files != []:
    bar1 = IncrementalBar(
        "Reseting check downloaded...",
        max=len(error_files),
        suffix="%(index)d/%(max)d | %(elapsed_td)s",
    )
    errors_id = [get_filename_extension(err)[0] for err in error_files]
    print(errors_id[:10])
    out_put = str(Path(ROOT, "check_download_match_id.parquet"))
    check_download_df = pd.read_parquet(out_put)
    for match_id in errors_id:
        

['6612710428', '6612710693', '6612711274', '6612711903', '6612712350', '6612713043', '6612714518', '6612718279', '6612718366', '6612720288']
   match_id  start_time  downloaded
0     12158  1314201270           0
1     12164  1314205112           0
2     12172  1314210111           0
3     12185  1314213494           0
4     12198  1314218033           0


In [ ]:
check_download_df.loc[check_download_df["match_id"] == match_id, "downloaded" ] = 0
        bar1.next()
    check_download_df.to_parquet(out_put, index=False)
    bar1.finish()
    # temp_df = pd.read_parquet(file_path)
    # # print(df.head())
    # check_download_df.loc[check_download_df["match_id"] == id, "downloaded"] = value
    print(check_download_df.head())
    # temp_df.to_parquet(file_path, index=False)



In [32]:
ms = 6612710428
check_download_df[check_download_df["match_id"] == 6612710428]

,match_id,start_time,downloaded
11791492,6612710428,1654957026,0


In [33]:
check_download_df.loc[check_download_df["match_id"] == ms, "downloaded" ] =0

In [34]:
check_download_df[check_download_df["match_id"] == 6612710428]

,match_id,start_time,downloaded
11791492,6612710428,1654957026,1


In [ ]:
# for k in data.keys():
#     print(str(k), len(data[k]))
match_df = pd.DataFrame(data)
# print(df.head())
account_df.to_parquet(CHECK_FILE, index=False)
match_df.to_parquet(OUTPUT_FILE, index=False)
print("Done.")

# df = pd.DataFrame(data_json)
# print(data)

# df = pd.read_json("crawled_data/matches/2022/7/10/6654586166.json")
# print(df.head())
if __name__ == "__main__":
    pass


In [28]:
import glob
import pandas as pd
import json
from progress.bar import IncrementalBar
def player_info_etl_dataframe():
    errors_list= []
    data_dict ={'solo_competitive_rank':[],	'leaderboard_rank':[], 'competitive_rank':[],'mmr_estimate':[],'profile':[],	'rank_tier':[]}
    columns = ['rank_tier','leaderboard_rank','profile','competitive_rank','mmr_estimate','solo_competitive_rank']
    players_info_list = glob.glob("D:\Python\hackathon\crawled_data\players_info\*.json")
    bar = IncrementalBar(
        "Extracting player...",
        max=len(players_info_list),
        suffix="%(index)d/%(max)d | %(elapsed_td)s",
    )
    print(len(players_info_list))
    for player_info in players_info_list:
        try:
            with open(player_info,"rb") as f:
                data = json.load(f)
                data_dict['solo_competitive_rank'].append(data['solo_competitive_rank'])
                data_dict['leaderboard_rank'].append(data['leaderboard_rank'])
                data_dict['competitive_rank'].append(data['competitive_rank'])
                data_dict['mmr_estimate'].append(data['mmr_estimate'])
                data_dict['profile'].append(data['profile'])
                data_dict['rank_tier'].append(data['rank_tier'])
            
        except Exception as e:
            errors_list.append(player_info)
        finally:
            bar.next()
    players_info_df =pd.DataFrame(data_dict)
    return players_info_df,errors_list
ka,err = player_info_etl_dataframe()
print(ka.head(10))
err
# df["haha"]

3328
   solo_competitive_rank  leaderboard_rank  competitive_rank  \
0                    NaN               NaN               NaN   
1                    NaN               NaN               NaN   
2                    NaN            4712.0               NaN   
3                    NaN               NaN               NaN   
4                 1161.0               NaN               NaN   
5                 3421.0               NaN               NaN   
6                 5954.0            5002.0            4922.0   
7                    NaN               NaN               NaN   
8                    NaN               NaN               NaN   
9                    NaN               NaN               NaN   

         mmr_estimate                                            profile  \
0  {'estimate': 3850}  {'account_id': 100009801, 'personaname': 'kill...   
1  {'estimate': 2932}  {'account_id': 1000479178, 'personaname': 'bag...   
2  {'estimate': 4544}  {'account_id': 1000737033, 'personaname

['D:\\Python\\hackathon\\crawled_data\\players_info\\100794739.json',
 'D:\\Python\\hackathon\\crawled_data\\players_info\\103940576.json',
 'D:\\Python\\hackathon\\crawled_data\\players_info\\1059062128.json',
 'D:\\Python\\hackathon\\crawled_data\\players_info\\1059586224.json',
 'D:\\Python\\hackathon\\crawled_data\\players_info\\109314489.json',
 'D:\\Python\\hackathon\\crawled_data\\players_info\\1102053806.json',
 'D:\\Python\\hackathon\\crawled_data\\players_info\\112853378.json',
 'D:\\Python\\hackathon\\crawled_data\\players_info\\114950606.json',
 'D:\\Python\\hackathon\\crawled_data\\players_info\\114950635.json',
 'D:\\Python\\hackathon\\crawled_data\\players_info\\115736750.json',
 'D:\\Python\\hackathon\\crawled_data\\players_info\\118625711.json',
 'D:\\Python\\hackathon\\crawled_data\\players_info\\121242023.json',
 'D:\\Python\\hackathon\\crawled_data\\players_info\\121766293.json',
 'D:\\Python\\hackathon\\crawled_data\\players_info\\122552810.json',
 'D:\\Python\\hac

In [40]:
import glob
import pandas as pd
import json
from progress.bar import IncrementalBar

def reset_download_error_files(check_file, errors_list:list, message="Reseting check downloaded of error files..."):
    if errors_list != []:
        bar = IncrementalBar(
           message,
            max=len(error_files),
            suffix="%(index)d/%(max)d | %(elapsed_td)s",
        )
        errors_id = [int(get_filename_extension(err)[0]) for err in error_files]
        # print(errors_id)
        out_put = check_file
        check_download_df = pd.read_parquet(out_put)
        for match_id in errors_id:
            check_download_df.loc[
                check_download_df["match_id"] == match_id, "downloaded"
            ] = 0
            bar.next()
        check_download_df.to_parquet(out_put, index=False)
        bar.finish()
    pass


def json_to_dataframe(input_path, columns ,message,check_file):
    """Return a pandas dataframe and list of error file path."""
    error_files= []
    # data_dict ={"win":[],"lose":[]}
    columns = ["win","lose"]
    data_dict = {col:[] for col in columns}

    # players_info_list = glob.glob("D:\Python\hackathon\crawled_data\players_wl\*.json")
    players_info_list = glob.glob(input_path)
    bar = IncrementalBar(message,
        max=len(players_info_list),
        suffix="%(index)d/%(max)d | %(elapsed_td)s",
    )
    print(len(players_info_list))
    for player_info in players_info_list:
        try:
            with open(player_info,"rb") as f:
                data = json.load(f)
                for col in columns:
                    data_dict[col].append(data[col])            
        except Exception as e:
            error_files.append(player_info)
        finally:
            bar.next()
    df =pd.DataFrame(data_dict)
    if errors_list == []:
        return True, df
    else:
        return False, error_files
columns = ["win","lose"]
ka,err = json_to_dataframe("D:\Python\hackathon\crawled_data\players_wl\*.json",columns,"Extracting players info...")
print(ka)
err

# data_dict = {col:[] for col in columns}


3327
False


['D:\\Python\\hackathon\\crawled_data\\players_wl\\100794739.json',
 'D:\\Python\\hackathon\\crawled_data\\players_wl\\103940576.json',
 'D:\\Python\\hackathon\\crawled_data\\players_wl\\1059062128.json',
 'D:\\Python\\hackathon\\crawled_data\\players_wl\\1059586224.json',
 'D:\\Python\\hackathon\\crawled_data\\players_wl\\109314489.json',
 'D:\\Python\\hackathon\\crawled_data\\players_wl\\1102053806.json',
 'D:\\Python\\hackathon\\crawled_data\\players_wl\\111542540.json',
 'D:\\Python\\hackathon\\crawled_data\\players_wl\\112853378.json',
 'D:\\Python\\hackathon\\crawled_data\\players_wl\\114950606.json',
 'D:\\Python\\hackathon\\crawled_data\\players_wl\\114950635.json',
 'D:\\Python\\hackathon\\crawled_data\\players_wl\\115736750.json',
 'D:\\Python\\hackathon\\crawled_data\\players_wl\\118625711.json',
 'D:\\Python\\hackathon\\crawled_data\\players_wl\\121242023.json',
 'D:\\Python\\hackathon\\crawled_data\\players_wl\\121766293.json',
 'D:\\Python\\hackathon\\crawled_data\\player